In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#### Imports

In [5]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import deque
from operator import itemgetter

In [3]:
np.random.seed(42)

#### Load the data

In [6]:
data_dir = '/home/users/dbernsohn/personal_storage/shopee'
train_img_dir = data_dir + os.sep + 'train_images'
df = pd.read_csv(f'{data_dir}{os.sep}train.csv')
df['path'] = train_img_dir + os.sep + df.image
df.head()

,posting_id,image,image_phash,title,label_group,path
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,/home/users/dbernsohn/personal_storage/shopee/...
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,/home/users/dbernsohn/personal_storage/shopee/...
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,/home/users/dbernsohn/personal_storage/shopee/...
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,/home/users/dbernsohn/personal_storage/shopee/...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,/home/users/dbernsohn/personal_storage/shopee/...


#### BKtree

In [7]:
__all__ = ['hamming_distance', 'BKTree']

__version__ = '1.1'

_getitem0 = itemgetter(0)

### BKTree
### source https://github.com/benhoyt/pybktree

class BKTree(object):
    """BK-tree data structure that allows fast querying of matches that are
    "close" given a function to calculate a distance metric (e.g., Hamming
    distance or Levenshtein distance).
    Each node in the tree (including the root node) is a two-tuple of
    (item, children_dict), where children_dict is a dict whose keys are
    non-negative distances of the child to the current item and whose values
    are nodes.
    """
    def __init__(self, distance_func, items=[]):
        self.distance_func = distance_func
        self.tree = None

        _add = self.add
        for item in items:
            _add(item)

    def add(self, item):
        node = self.tree
        if node is None:
            self.tree = (item, {})
            return

        # Slight speed optimization -- avoid lookups inside the loop
        _distance_func = self.distance_func

        while True:
            parent, children = node
            distance = _distance_func(item, parent)
            node = children.get(distance)
            if node is None:
                children[distance] = (item, {})
                break

    def find(self, item, n):
        if self.tree is None:
            return []

        candidates = deque([self.tree])
        found = []

        # Slight speed optimization -- avoid lookups inside the loop
        _candidates_popleft = candidates.popleft
        _candidates_extend = candidates.extend
        _found_append = found.append
        _distance_func = self.distance_func

        while candidates:
            candidate, children = _candidates_popleft()
            distance = _distance_func(candidate, item)
            if distance <= n:
                _found_append((distance, candidate))

            if children:
                lower = distance - n
                upper = distance + n
                _candidates_extend(c for d, c in children.items() if lower <= d <= upper)

        found.sort(key=_getitem0)
        return found

    def __iter__(self):
        if self.tree is None:
            return

        candidates = deque([self.tree])

        # Slight speed optimization -- avoid lookups inside the loop
        _candidates_popleft = candidates.popleft
        _candidates_extend = candidates.extend

        while candidates:
            candidate, children = _candidates_popleft()
            yield candidate
            _candidates_extend(children.values())

    def __repr__(self):
        return '<{} using {} with {} top-level nodes>'.format(
            self.__class__.__name__,
            self.distance_func.__name__,
            len(self.tree[1]) if self.tree is not None else 'no',
        )

In [8]:
def hex_to_hash(hexstr):
    # modified function from imagehash
    # https://github.com/JohannesBuchner/imagehash/blob/d7dffec20dcfc7d1afac90ad7d447cd06240087c/imagehash.py
    
    hash_size = int(np.sqrt(len(hexstr)*4))
    #assert hash_size == np.sqrt(len(hexstr)*4)
    binary_array = '{:0>{width}b}'.format(int(hexstr, 16), width = hash_size * hash_size)
    bit_rows = [binary_array[i:i+hash_size] for i in range(0, len(binary_array), hash_size)]
    hash_array = np.array([[bool(int(d)) for d in row] for row in bit_rows])
    return hash_array.flatten().astype(int)

def simple_hamming(x1, x2, idx =0):
    #hamming distance between integer arrays
    return np.count_nonzero(x1[idx]!=x2[idx])

df['phash_vec'] = df['image_phash'].apply(hex_to_hash)

tree_items = list(df[['phash_vec', 'label_group', 'title', 'image', 'posting_id']].values)

tree = BKTree(simple_hamming, tree_items)

# consider using different threshold
hamming_distance = 1

suspicious_groups = []

# O(nlogn)
for i in tqdm(range(len(tree_items))):
    
    neighbor_list = tree.find(tree_items[i], hamming_distance)
    
    if len(neighbor_list)>1:
        
        neighbor_labels = [x[1][1] for x in neighbor_list]
    
        # if all items in the neighbor_list don't have the same label then we assume
        # that some samples have wrong label
        
        if len(set(neighbor_labels))>1:
            suspicious_groups.append(neighbor_list)

In [9]:
sus_ids = []
for group in suspicious_groups:
    for item in group:
        sus_ids.append(item[1][4])

n_sus = len(set(sus_ids))
print('Total number of suspicious samples: {} or in percent: {}%'.format(n_sus, round(n_sus/len(df)*100,2)))
print('Consider removing them from the training set')

Total number of suspicious samples: 468 or in percent: 1.37%
Consider removing them from the training set


In [10]:
print(df.shape[0])
df = df[~df.posting_id.isin(sus_ids)]
print(df.shape[0])

34250
33782


In [ ]:
df.to_csv('filtered_train.csv')